In [1]:
import pandas as pd
import numpy as np
from ortools.sat.python import cp_model
from datetime import datetime, timedelta

In [2]:
import sys
sys.path.append('/home/cayena/Projeto_thefamily/TeacherScheduler/')

In [3]:
from streamlit_app.utils import transform_classes_dateframe,transform_teacher_dataframe,filter_class_without_teacher,transform_alocation_dataframe
from streamlit_app.teacher_alocation import TeacherScheduler

In [4]:
professores_raw = pd.read_excel('/home/cayena/Downloads/professores_final.xlsx')
aulas_raw = pd.read_excel('/home/cayena/Downloads/MODELO TESTE CARLOS.xlsx')

In [5]:
df_result =transform_classes_dateframe(aulas_raw)
df_professor = transform_teacher_dataframe(professores_raw)

In [8]:
Ts = TeacherScheduler(df_result, professores_raw)
base_alocada = Ts.schedule_teachers(seed=93474)
base_alocada

93474


,professores_alocados,nome grupo
0,NICK,ABU DHABI EAD - 4X
1,MARIA,AUCKLAND P - 4X
2,LUCIANO,BAKERSFIELD P - 4X
3,TATIANA ERBEN,BERMUDAS P - 4X
4,NATACHA,BIG BEAR LAKE EAD - 2X
...,...,...
113,BARBARA,NEBRASKA P - 3X
114,FERNANDO,NORTHAMPTOM P - 3X
115,MARIA,SAN JUAN EAD - 3X
116,CLAUDIO,SCOTT EAD - 3X


In [9]:
df_final,aulas_nao_alocadas = transform_alocation_dataframe(aulas_raw,base_alocada)
df_final

,nome grupo,horario,unidade,dias da semana,stage,modalidade,grupo,n aulas,teacher,status,restricoes_professor,ultimo_professor,penultimo_professor,STAGE,PARAG FINAL,nº 1 sempre
0,ABU DHABI EAD - 4X,19:00:00,Vicentina,2ª ● 3ª ● 4ª ● 5ª,11,Inglês,Grupo,4.0,NICK,ONLINE,NaN,NICK,NaN,11,1246,1
1,ANDERSON VIP EAD - 1X,15:30:00,Vicentina,6ª,2,Inglês,VIP,1.0,BRUNO MORGILLO,ONLINE,NaN,BRUNO MORGILLO,NaN,2,170,1
2,ANDRE VIP CONV EAD - 2X,07:30:00,Vicentina,4ª ● 6ª,CONV,CONV - Ing Avançado,VIP,2.0,BRUNO MORGILLO,ONLINE,NaN,BRUNO MORGILLO,NaN,CONV,CONV,1
3,AUCKLAND P - 4X,19:00:00,Satélite,2ª ● 3ª ● 4ª ● 5ª,4,Inglês,Grupo,4.0,MARIA,PRESENCIAL,NaN,MARIA,NaN,4,390,1
4,AUSTIN EAD - 3X,08:00:00,Satélite,Saturday - Triple,4,Inglês,Grupo,3.0,ANDREY,ONLINE,NaN,ANDREY,NaN,4,390,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,WARRINGTON P - 4X,18:00:00,Satélite,2ª ● 3ª ● 4ª ● 5ª,2,Inglês,Grupo,4.0,CENKAY,PRESENCIAL,NaN,CENKAY,NaN,2,170,1
134,WILLIAMSBURGH P - 3X,19:00:00,Satélite,2ª ● 3ª ● 5ª,6,Inglês,Grupo,3.0,DUDU,PRESENCIAL,NaN,DUDU,NaN,6,624,1
135,WISCONSIN EAD - 3X,20:00:00,Jardim,2ª ● 4ª ● 5ª,2,Inglês,Grupo,3.0,ANDREY,ONLINE,NaN,ANDREY,NaN,2,170,1
136,WYOMING P - 4X,19:00:00,Satélite,2ª ● 3ª ● 4ª ● 5ª,9,Inglês,Grupo,4.0,DARLENE,PRESENCIAL,NaN,DARLENE,NaN,9,990,1


In [10]:
aulas_nao_alocadas

,nome grupo,horario,unidade,dias da semana,stage,modalidade,grupo,n aulas,teacher,status,restricoes_professor,ultimo_professor,penultimo_professor,STAGE,PARAG FINAL,nº 1 sempre
6,BARCELONA P - 3X,08:00:00,Satélite,Saturday - Triple,3,Espanhol,Grupo,3.0,-,PRESENCIAL,NaN,-,NaN,3,301,1
